# Constructing the likelihood of cosmological parameters using MCMC chain:
# Metropolis-Hastings vs. Affine-Invariance


In this notebook, I show how MCMC chains work to evaluate the likelihood of cosmological parameters

The cosmic microwave background (CMB) is electromagnetic radiation as a remnant from an early stage of the universe in the standard cosmology. The use of CMB observation enables us to explore the evolution of the Universe and temperatures at all angular positions are measured for this study. There are 6 parameters which descibe the evolution of the cosmic structure and the paramters with relavant physical models constraint the angular distribution of temperature. The cosmological parameters are:
> - **Ω_b**: density of baryonic matter
- **Ω_cdm**: density of cold dark matter (cdm)
- **τ_reio**: optical depth at the epoch of reionization
- **θ_s**: the angular scale of the sound horizon at matter-radiation equality
- **A_s**: curvature perturbations
- **n_s**: scalar spectral index.



The finite-dimensional distribution (hearafter f.d.d. data is discretely spaced in the units of pixel) of cosmic temperature follow a Gaussian random process. The observed data consist of: 
\begin{aligned}
\vec{\rm d}~({\rm =}~ d(\hat{n})) = T(\hat{n}) + \mathcal{E}(\hat{n}) 
\end{aligned}
where $\vec{\rm d}$(or d($\hat{n}$)) is data , T($\hat{n}$) $\sim (0,\Sigma^{TT}(\vec{p}))$ is model temperature, $\mathcal{E}$($\hat{n}$) $\sim (0,\Sigma^{\mathcal{EE}})$ is noise bias, and $\hat{n}$ is angular position vector. 

Acutally, the temperature we observe is PSF-convolved one. Therefore, <br>
\begin{aligned}
&~d(\hat{n}) = \phi * T(\hat{n}) + \mathcal{E}(\hat{n}) ~ :{\rm general~expression}\\
\Longleftrightarrow &~d_{lm} = (\phi * T)_{lm} + \mathcal{E}_{lm} ~~~ :{\rm expressed~in~spherical~harmonics~on~S}^2
\end{aligned}              
where $\phi$ is a Gaussian beam PSF on $S^2$.

Dividing by the beam and renaming $d_{lm}~\&~\mathcal{E}_{lm}$ gives a general model for observation:
\begin{aligned}
d_{lm} = T_{lm} + \mathcal{E}_{lm} ~~~ {\rm for~}l={\rm 0,1,2,\cdots,~and~}m=-l,-l+1,\cdots,l-1,l
\end{aligned}     
where $E(T_{lm}\cdot\overline{T_{l'm'}})=\delta_{ll'}\delta_{mm'}C_l^{TT}$,
$E(\mathcal{E}_{lm}\cdot\overline{\mathcal{E}_{l'm'}})=\delta_{ll'}\delta_{mm'}C_l^{\mathcal{EE}}
= \delta_{ll'}\delta_{mm'}\sigma^2\exp\bigg(\frac{b^2}{8\log 2}l(l+1)\bigg)$, $\sigma$ is addictive noise level, and $b$ is FWHM of the PSF.



# Getting Started: reading data & initializing paramters

As the first step of our analysis, I read a set of 9-year WMAP data with estimated cosmological paramters and fully available  MCMC chain (Hinshaw et al. 2013). The mean of each parameter data is the best estimate by WMAP observation.

In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import h5py
import sys
import warnings
import pypico
warnings.filterwarnings('ignore')
path = '/Users/inchanji/Research/wmap_lcdm_wmap9_chains_v5/'

In [2]:
nlines = sum(1 for line in open(path+'omegach2'))
nchain = 100_000

omega_b_chain     = np.loadtxt(path + 'omegabh2', usecols=(1),dtype = np.float64)[:nchain]
omega_cdm_chain   = np.loadtxt(path + 'omegach2', usecols=(1),dtype = np.float64)[:nchain]
tau_reio_chain    = np.loadtxt(path + 'tau', usecols=(1),dtype = np.float64)[:nchain]
theta_s_chain     = np.loadtxt(path + 'thetastar', usecols=(1),dtype = np.float64)[:nchain]
A_s_109_chain     = np.loadtxt(path + 'a002', usecols=(1),dtype = np.float64)[:nchain]
# which is 10⁹ * A_s
n_s_chain         = np.loadtxt(path + 'ns002', usecols=(1),dtype = np.float64)[:nchain]


In [3]:
full_chain = np.stack((omega_b_chain, omega_cdm_chain, 
                       tau_reio_chain, theta_s_chain, 
                       A_s_109_chain,n_s_chain), axis = 1)

wmap_chain = full_chain.copy()

In [4]:
sig2 = (10.0/3437.75)**2  # <---10μk arcmin noise level converted to radian pixels (1 radian = 3437.75 arcmin)
b2   = 0.0035**2          # <-- FWHM of 0.2ᵒ ≈ 12.0 armin ≈ 0.0035 radians          

In [5]:
def cov(mat):
    # kernel is dead when using np.cov()
    _, ncov = np.shape(mat)
    covmat = np.zeros((ncov,ncov))
    for i in range(ncov):
        for j in range(i,ncov):
            if i==j:
                covmat[i,j] = np.var(mat[:,i])
            else:
                covmat[i,j] = np.cov(mat[:,i],mat[:,j])[0,1]
                covmat[j,i] = np.cov(mat[:,i],mat[:,j])[1,0]
    return covmat

In [6]:
cov_wmap = cov(full_chain)

In [7]:
f = h5py.File(path + 'bandpowers.h5', 'r')
list(f.keys())

['bandpowers']

In [8]:
bandpowers = f['bandpowers']
np.shape(bandpowers)

(5626,)

In [9]:
f = h5py.File(path + 'lcdm_sim_truth.h5', 'r')
list(f.keys())

['lcdm_sim_truth']

In [10]:
lcdm_sim_truth = f['lcdm_sim_truth']
np.shape(lcdm_sim_truth)

(6, 1)

In [11]:
lcdm_sim_truth.value

array([[0.02154707],
       [0.11744188],
       [0.0908262 ],
       [0.01040136],
       [2.57248823],
       [0.95556754]])

In [18]:
picoload = pypico.load_pico(path+'pico3_tailmonty_v34_py3.dat')

In [19]:
picoload.inputs()

{'Alens',
 'helium_fraction',
 'massive_neutrinos',
 'ombh2',
 'omch2',
 'omnuh2',
 'pivot_scalar',
 're_optical_depth',
 'scalar_amp(1)',
 'scalar_nrun(1)',
 'scalar_spectral_index(1)',
 'theta'}

In [29]:
dict_input = {'scalar_amp(1)': 1e-9*A_s_109_chain[0],
             're_optical_depth':tau_reio_chain[0],
             'theta': theta_s_chain[0],
             'ombh2': omega_b_chain[0],
             'omch2': omega_cdm_chain[0],
             'scalar_spectral_index(1)': n_s_chain[0],
             'massive_neutrinos': 3.046,
             'helium_fraction':0.248, 
             'omnuh2': 0, 
             'scalar_nrun(1)':0
             }
plout = picoload.get(**dict_input)

In [31]:
plout['cl_TT']

array([1.16099661e+03, 1.13585167e+03, 1.08457830e+03, ...,
       2.62957422e-01, 2.62668415e-01, 2.62379909e-01])

In [30]:
print(picoload.example_inputs())
print(picoload.outputs())

{'helium_fraction': 0.248, 'massive_neutrinos': 3.046, 'ombh2': 0.022, 'omch2': 0.125, 'omnuh2': 0, 're_optical_depth': 0.085, 'scalar_amp(1)': 2.5e-09, 'scalar_nrun(1)': 0, 'scalar_spectral_index(1)': 0.97, 'theta': 0.0104, 'pivot_scalar': 0.002}
['cl_TT', 'cl_TE', 'cl_EE', 'cl_BB', 'k', 'lensed_TT', 'scalar_EE', 'scalar_TT', 'cl_pT', 'lensed_EE', 'lensed_BB', 'pk', 'scalar_TE', 'lensed_TE', 'cl_pp']


In [37]:
dict_input['scalar_amp(1)'][0]

2.4353973412148978e-09

In [38]:
1e-9*A_s_109_chain

array([2.43539734e-09, 2.28535823e-09, 2.29472634e-09, ...,
       2.31519636e-09, 2.32093190e-09, 2.45407195e-09])

In [41]:
dict_input.keys()

dict_keys(['scalar_amp(1)', 're_optical_depth', 'theta', 'ombh2', 'omch2', 'scalar_spectral_index(1)', 'massive_neutrinos', 'helium_fraction', 'omnuh2', 'scalar_nrun(1)'])

In [42]:
picoload._data

{'fits': {'lensed_TT': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x10ed7b5c0>],
  'scalar_EE': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x10ed750b8>],
  'scalar_TT': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x10ed75048>],
  'cl_pT': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x1a1cec30b8>],
  'lensed_EE': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x1a1cec3128>],
  'lensed_BB': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x1a1cec3208>],
  'pk': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x1a1cec32e8>],
  'scalar_TE': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x1a1cec33c8>],
  'lensed_TE': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x1a1cec3438>],
  'cl_pp': [<pypico.datafiles.3e624ff23f85df8ba6355a4e18135245.PicoFit at 0x1a1cec3518>]},
 'k': array([9.990000e-05, 1.046329e-04, 1.095900e-04, 1.147819e-04,
